# CMU 10-605/10-805 auto-graded notebook

Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

---

# Homework 3: Logistic Regression

In [ ]:
# Who did you collaborate with on this assignment? 
# if no one, collaborators should contain an empty string,
# else list your collaborators below

# collaborators = [""]
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
try:
    collaborators
except:
    raise AssertionError("you did not list your collaborators, if any")

# Click-Through Rate Prediction
In this section you will go through the steps for creating a click-through rate (CTR) prediction pipeline. You will work with the Criteo Labs dataset.

## This section will cover:

* *Part 1:* Featurize categorical data using one-hot-encoding (OHE)

* *Part 2:* Construct an OHE dictionary

* *Part 3:* Parse CTR data and generate OHE features
 * *Visualization 1:* Feature frequency

* *Part 4:* CTR prediction and logloss evaluation
 * *Visualization 2:* ROC curve

* *Part 5:* Reduce feature dimension via feature hashing

> Note that, for reference, you can look up the details of:
> * the relevant Spark methods in [PySpark's DataFrame API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.DataFrame)
> * the relevant NumPy methods in the [NumPy Reference](http://docs.scipy.org/doc/numpy/reference/index.html)

In [ ]:
from nose.tools import assert_equal, assert_true

In [ ]:
# YOU CAN MOST LIKELY IGNORE THIS CELL. This is only of use for running this notebook locally.

# THIS CELL DOES NOT NEED TO BE RUN ON DATABRICKS. 
# Note that Databricks already creates a SparkContext for you, so this cell can be skipped.
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext(appName="hw")
sqlContext = SQLContext(sc)

print("spark context started")

## Part 1: Featurize categorical data using one-hot-encoding

### (1a) One-hot-encoding

We would like to develop code to convert categorical features to numerical ones. In order to build intuition, we will work with a unlabeled dataset with three data points, with each data point representing an animal. The first feature indicates the type of animal (bear, cat, mouse); the second feature describes the animal's color (black, tabby); and the third (optional) feature describes what the animal eats (mouse, salmon).

In a one-hot-encoding (OHE) scheme, we want to represent each tuple of `(featureID, category)` via its own binary feature.  We can do this in Python by creating a dictionary that maps each tuple to a distinct integer, where the integer corresponds to a binary feature. To start, manually enter the entries in the OHE dictionary associated with the sample dataset by mapping the tuples to consecutive integers starting from zero,  ordering the tuples first by featureID and next by category.

Later in this lab, we'll use OHE dictionaries to transform data points into compact lists of features that can be used in machine learning algorithms.

In [ ]:
# By default, when a shuffle operation occurs with DataFrames, the post-shuffle partition
# count is 200. This is controlled by Spark configuration value spark.sql.shuffle.partitions.
# 200 is a little too high for this data set, so we set the post-shuffle partition count to
# twice the number of available threads in Community Edition.
sqlContext.setConf('spark.sql.shuffle.partitions', '6')  # Set default partitions for DataFrame operations

In [ ]:
from collections import defaultdict
# Data for manual OHE
# Note: the first data point does not include any value for the optional third feature
sample_one = [(0, 'mouse'), (1, 'black')]
sample_two = [(0, 'cat'), (1, 'tabby'), (2, 'mouse')]
sample_three =  [(0, 'bear'), (1, 'black'), (2, 'salmon')]

def sample_to_row(sample):
    tmp_dict = defaultdict(lambda: None)
    tmp_dict.update(sample)
    return [tmp_dict[i] for i in range(3)]

sqlContext.createDataFrame(map(sample_to_row, [sample_one, sample_two, sample_three]),
                           ['animal', 'color', 'food']).show()
sample_data_df = sqlContext.createDataFrame([(sample_one,), (sample_two,), (sample_three,)], ['features'])
sample_data_df.show(truncate=False)

In [ ]:
# # TODO: Replace <FILL IN> with appropriate code
# sample_ohe_dict_manual = {}
# sample_ohe_dict_manual[(0, 'bear')] = <FILL IN >
# sample_ohe_dict_manual[(0, 'cat')] = <FILL IN >
# sample_ohe_dict_manual[(0, 'mouse')] = <FILL IN >
# sample_ohe_dict_manual < FILL IN >
# sample_ohe_dict_manual < FILL IN >
# sample_ohe_dict_manual < FILL IN >
# sample_ohe_dict_manual < FILL IN >

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST One-hot-encoding (1a)
assert_equal(sample_ohe_dict_manual[(0, 'bear')], 0)
assert_equal(sample_ohe_dict_manual[(0, 'cat')], 1)
assert_equal(sample_ohe_dict_manual[(0, 'mouse')], 2)
assert_equal(len(sample_ohe_dict_manual.keys()), 7)

### (1b) Sparse vectors

Data points can typically be represented with a small number of non-zero OHE features which are relative to the total number of features that occur in the dataset.  By leveraging this sparsity and using sparse vector representations for OHE data, we can reduce storage and computational burdens.  Below are a few sample vectors represented as dense numpy arrays.  Use [SparseVector](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector) to represent them in a sparse fashion, and verify that both the sparse and dense representations yield the same results when computing [dot products](http://en.wikipedia.org/wiki/Dot_product) (we will later use MLlib to train classifiers via gradient descent, and MLlib will need to compute dot products between SparseVectors and dense parameter vectors).

Use `SparseVector(size, *args)` to create a new sparse vector where size is the length of the vector and args are either:
1. A list of indices and a list of values corresponding to the indices. The indices list must be sorted in ascending order. For example, SparseVector(5, [1, 3, 4], [10, 30, 40]) will represent the vector [0, 10, 0, 30, 40]. The non-zero indices are 1, 3 and 4. On the other hand, SparseVector(3, [2, 1], [5, 5]) will give you an error because the indices list [2, 1] is not in ascending order. Note: you cannot simply sort the indices list, because otherwise the values will not correspond to the respective indices anymore.
2. A list of (index, value) pair. In this case, the indices need not be sorted. For example, SparseVector(5, [(3, 1), (1, 2)]) will give you the vector [0, 2, 0, 1, 0].

SparseVectors are much more efficient when working with sparse data because they do not store zero values (only store non-zero values and their indices). You'll need to create a sparse vector representation of each dense vector `a_dense` and `b_dense`.

In [ ]:
import numpy as np
from pyspark.ml.linalg import SparseVector

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# a_dense = np.array([0., 3., 0., 4.])
# a_sparse = <FILL IN >

# b_dense = np.array([0., 0., 0., 1.])
# b_sparse = <FILL IN >

# YOUR CODE HERE
raise NotImplementedError()

w = np.array([0.4, 3.1, -1.4, -.5])
print(a_dense.dot(w))
print(a_sparse.dot(w))
print(b_dense.dot(w))
print(b_sparse.dot(w))

In [ ]:
# TEST Sparse Vectors (1b)
assert_true(isinstance(a_sparse, SparseVector), 'a_sparse needs to be an instance of SparseVector')
assert_true(b_dense.dot(w) == b_sparse.dot(w),
                'dot product of b_dense and w should equal dot product of b_sparse and w')
assert_true(a_sparse.numNonzeros() == 2, 'a_sparse should not store zero values')

### (1c) OHE features as sparse vectors

Now let's see how we can represent the OHE features for points in our sample dataset.  Using the mapping defined by the OHE dictionary from Part (1a), manually define OHE features for the three sample data points using SparseVector format.  In this case, all the features will have a value of 1.0.  For example, the `DenseVector` for a point with features 2 and 4 would be `[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]`.

In [ ]:
# Reminder of the sample features
# sample_one = [(0, 'mouse'), (1, 'black')]
# sample_two = [(0, 'cat'), (1, 'tabby'), (2, 'mouse')]
# sample_three =  [(0, 'bear'), (1, 'black'), (2, 'salmon')]

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code. Use SparseVector
# sample_one_ohe_feat_manual = <FILL IN >
# sample_two_ohe_feat_manual = <FILL IN >
# sample_three_ohe_feat_manual = <FILL IN >

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST OHE Features as sparse vectors (1c)
assert_true(isinstance(sample_one_ohe_feat_manual, SparseVector),
                'sample_one_ohe_feat_manual needs to be a SparseVector')
assert_true(isinstance(sample_two_ohe_feat_manual, SparseVector),
                'sample_two_ohe_feat_manual needs to be a SparseVector')
assert_true(isinstance(sample_three_ohe_feat_manual, SparseVector),
                'sample_three_ohe_feat_manual needs to be a SparseVector')

assert_equal(sample_one_ohe_feat_manual[2], 1.0, 'incorrect value for sample_one_ohe_feat_manual')
assert_equal(sample_one_ohe_feat_manual[3], 1.0, 'incorrect value for sample_one_ohe_feat_manual')


### (1d) Define a OHE function

Next we will use the OHE dictionary from Part (1a) to programatically generate OHE features from the original categorical data.  First write a function called `one_hot_encoding` that creates OHE feature vectors in `SparseVector` format.  Then use this function to create OHE features for the first sample data point and verify that the result matches the result from Part (1c).

> Note: We'll pass in the OHE dictionary as a [Broadcast](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.Broadcast) variable, which will greatly improve performance when we call this function as part of a UDF. **When accessing a broadcast variable, you _must_ use `.value`.** For instance: `ohe_dict_broadcast.value`.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def one_hot_encoding(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    # <FILL IN>
    # ohe_features = [<FILL IN>]
    # return SparseVector(<FILL IN>)

    # YOUR CODE HERE
    raise NotImplementedError()
  
# # Calculate the number of features in sample_ohe_dict_manual
# num_sample_ohe_feats = <FILL IN >
# sample_ohe_dict_manual_broadcast = sc.broadcast(sample_ohe_dict_manual)

# # Run one_hot_encoding() on sample_one.  Make sure to pass in the Broadcast variable.
# sample_one_ohe_feat = <FILL IN >  
  
# YOUR CODE HERE
raise NotImplementedError()

print(sample_one_ohe_feat)

In [ ]:
# TEST Define an OHE Function (1d)
assert_true(sample_one_ohe_feat == sample_one_ohe_feat_manual,
                'sample_one_ohe_feat should equal sample_one_ohe_feat_manual')
assert_equal(sample_one_ohe_feat, SparseVector(7, [2, 3], [1.0, 1.0]),
                  'incorrect value for sample_one_ohe_feat')
assert_equal(one_hot_encoding([(1, 'black'), (0, 'mouse')], sample_ohe_dict_manual_broadcast,
                                   num_sample_ohe_feats), SparseVector(7, [2, 3], [1.0, 1.0]),
                  'incorrect definition for one_hot_encoding')

### (1e) Apply OHE to a dataset

Finally, use the function from Part (1d) to create OHE features for all 3 data points in the sample dataset.  You'll need to generate a [UDF](https://spark.apache.org/docs/1.6.1/api/python/pyspark.sql.html#pyspark.sql.functions.udf) that can be used in a `DataFrame` `select` statement.

> Note: Your implemenation of `ohe_udf_generator` needs to call your `one_hot_encoding` function.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT

def ohe_udf_generator(ohe_dict_broadcast):
    """Generate a UDF that is setup to one-hot-encode rows with the given dictionary.

    Note:
        We'll reuse this function to generate a UDF that can one-hot-encode rows based on a
        one-hot-encoding dictionary built from the training data.  Also, you should calculate
        the number of features before calling the one_hot_encoding function.

    Args:
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.

    Returns:
        UserDefinedFunction: A UDF can be used in `DataFrame` `select` statement to call a
            function on each row in a given column.  This UDF should call the one_hot_encoding
            function with the appropriate parameters.
    """
#     length = <FILL IN>
#     return udf(lambda x: <FILL IN>, VectorUDT())

# YOUR CODE HERE
raise NotImplementedError()

# sample_ohe_dict_udf = ohe_udf_generator(sample_ohe_dict_manual_broadcast)
# sample_ohe_df = sample_data_df.select( < FILL IN >)
# sample_ohe_df.show(truncate=False)

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST Apply OHE to a dataset (1e)
sample_ohe_data_values = sample_ohe_df.collect()
assert_true(len(sample_ohe_data_values) == 3, 'sample_ohe_data_values should have three elements')
assert_equal(sample_ohe_data_values[0], (SparseVector(7, {2: 1.0, 3: 1.0}),),
                  'incorrect OHE for first sample')
assert_equal(sample_ohe_data_values[1], (SparseVector(7, {1: 1.0, 4: 1.0, 5: 1.0}),),
                  'incorrect OHE for second sample')
assert_equal(sample_ohe_data_values[2], (SparseVector(7, {0: 1.0, 3: 1.0, 6: 1.0}),),
                  'incorrect OHE for third sample')

## Part 2: Construct an OHE dictionary

### (2a) DataFrame with rows of `(featureID, category)`

To start, create a DataFrame of distinct `(feature_id, category)` tuples. In our sample dataset, the 7 items in the resulting DataFrame are `(0, 'bear')`, `(0, 'cat')`, `(0, 'mouse')`, `(1, 'black')`, `(1, 'tabby')`, `(2, 'mouse')`, `(2, 'salmon')`. Notably `'black'` appears twice in the dataset but only contributes one item to the DataFrame: `(1, 'black')`, while `'mouse'` also appears twice and contributes two items: `(0, 'mouse')` and `(2, 'mouse')`.  Use [explode](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode) and [distinct](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct).

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
from pyspark.sql.functions import explode
# sample_distinct_feats_df = (sample_data_df
#                               <FILL IN>)
# YOUR CODE HERE
raise NotImplementedError()
sample_distinct_feats_df.show()

In [ ]:
# TEST DataFrame with rows of `(featureID, category)` (2a)
assert_equal(sorted(map(lambda r: r[0], sample_distinct_feats_df.collect())),
                  [(0, 'bear'), (0, 'cat'), (0, 'mouse'), (1, 'black'),
                   (1, 'tabby'), (2, 'mouse'), (2, 'salmon')],
                  'incorrect value for sample_distinct_feats_df')

### (2b) OHE Dictionary from distinct features

Next, create an RDD of key-value tuples, where each `(feature_id, category)` tuple in `sample_distinct_feats_df` is a key and the values are distinct integers ranging from 0 to (number of keys - 1).  Then convert this RDD into a dictionary, which can be done using the `collectAsMap` action.  Note that there is no unique mapping from keys to values, as all we require is that each `(featureID, category)` key be mapped to a unique integer between 0 and the number of keys.  In this exercise, any valid mapping is acceptable.  Use [zipWithIndex](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.zipWithIndex) followed by [collectAsMap](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collectAsMap).

In our sample dataset, one valid list of key-value tuples is: `[((0, 'bear'), 0), ((2, 'salmon'), 1), ((1, 'tabby'), 2), ((2, 'mouse'), 3), ((0, 'mouse'), 4), ((0, 'cat'), 5), ((1, 'black'), 6)]`. The dictionary defined in Part (1a) illustrates another valid mapping between keys and integers.

> Note: We provide the code to convert the DataFrame to an RDD.

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# sample_ohe_dict = (sample_distinct_feats_df
#                      .rdd
#                      .map(lambda r: tuple(r[0]))
#                      <FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()
print(sample_ohe_dict)

In [ ]:
# TEST OHE Dictionary from distinct features (2b)
assert_equal(sorted(sample_ohe_dict.keys()),
                  [(0, 'bear'), (0, 'cat'), (0, 'mouse'), (1, 'black'),
                   (1, 'tabby'), (2, 'mouse'), (2, 'salmon')],
                  'sample_ohe_dict has unexpected keys')
assert_equal(sorted(sample_ohe_dict.values()), list(range(7)), 'sample_ohe_dict has unexpected values')

### (2c) Automated creation of an OHE dictionary

Now use the code from Parts (2a) and (2b) to write a function that takes an input dataset and outputs an OHE dictionary.  Then use this function to create an OHE dictionary for the sample dataset, and verify that it matches the dictionary from Part (2b).

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def create_one_hot_dict(input_df):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        input_df (DataFrame with 'features' column): A DataFrame where each row contains a list of
            (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
#    <FILL IN>
#    distinct_feature_df = (<FILL IN>)
#    key_value_tuple_list = (<FILL IN)
#    return dict(key_value_tuple_list)

# YOUR CODE HERE
raise NotImplementedError()

sample_ohe_dict_auto = create_one_hot_dict(sample_data_df)

In [ ]:
# TEST Automated creation of an OHE dictionary (2c)
assert_equal(sorted(sample_ohe_dict_auto.keys()),
                  [(0, 'bear'), (0, 'cat'), (0, 'mouse'), (1, 'black'),
                   (1, 'tabby'), (2, 'mouse'), (2, 'salmon')],
                  'sample_ohe_dict_auto has unexpected keys')
assert_equal(sorted(sample_ohe_dict_auto.values()), list(range(7)),
                  'sample_ohe_dict_auto has unexpected values')

## Part 3: Parse CTR data and generate OHE features

***Before we can proceed, you'll first need to obtain the sample data.  ***

The data is from a past [kaggle competition](https://www.kaggle.com/c/criteo-display-ad-challenge/overview). The original data was too big. So we randomly samepled the data for this assignment from the original dataset.

For data fileds:
* Label - Target variable that indicates if an ad was clicked (1) or not (0).
* I1-I13 - A total of 13 columns of integer features (mostly count features).
* C1-C26 - A total of 26 columns of categorical features. The values of these features have been hashed onto 32 bits for anonymization purposes.
The semantic of the features is undisclosed.
Just run the cell below.

In [ ]:
# Run this cell for getting data from the github repo.
from pyspark import SparkFiles
from pyspark.sql import Row
url = "https://raw.githubusercontent.com/10605/data/master/hw3/dac.txt"
sc.addFile(url)

raw_df = sc.textFile("file://" + SparkFiles.get("dac.txt")).map(lambda r: Row(r)).toDF(["text"])

In [ ]:
raw_df.show()

### (3a) Loading and splitting the data

We are now ready to start working with the actual CTR data, and our first task involves splitting it into training, validation, and test sets.  Usually we use the [randomSplit method](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) with the specified weights and seed to create DFs storing each of these datasets. BUT randomSplit may generate non-deterministic results. So for the sake of testing, we manually split the data in to train, validation, test by the ratio of 0.8, 0.1, 0.1.

Then your work is to [cache](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cache) each of these DFs, as we will access them multiple times in the remainder of this lab. Finally, compute the size of each dataset.

In [ ]:
import pyspark.sql.functions as f
indexDf = raw_df.withColumn('index', f.monotonically_increasing_id())
total_count = raw_df.count()
train_count = int(total_count * 4 / 5)
dev_count = int(total_count / 5)
val_count = int(total_count / 10)
test_count = int(total_count / 10)
# first 20% rows
raw_dev_df = indexDf.sort('index').limit(dev_count)
# last 80% rows
raw_train_df = indexDf.sort('index', ascending=False).limit(train_count).drop('index')
# first 10% rows
raw_validation_df = raw_dev_df.sort('index').limit(val_count).drop('index')
# last 10% rows
raw_test_df = raw_dev_df.sort('index', ascending=False).limit(test_count).drop('index')

# # Cache and count the DataFrames
# n_train = raw_train_df.<FILL IN>
# n_val = raw_validation_df.<FILL IN>
# n_test = raw_test_df.<FILL IN>

# YOUR CODE HERE
raise NotImplementedError()
print(n_train, n_val, n_test, n_train + n_val + n_test)
raw_df.show(1)

In [ ]:
# TEST Loading and splitting the data (3a)
assert_true(all([raw_train_df.is_cached, raw_validation_df.is_cached, raw_test_df.is_cached]),
                'you must cache the split data')
assert_equal(n_train, 80000, 'incorrect value for n_train')
assert_equal(n_val, 10000, 'incorrect value for n_val')
assert_equal(n_test, 10000, 'incorrect value for n_test')

### (3b) Extract features

We will now parse the raw training data in order to create a DataFrame that we can subsequently use to create an OHE dictionary. Note from the `show()` command in Part (3a) that each raw data point is a string containing several fields separated by some delimiters.  For now, we will ignore the first field (which is just the 0-1 label), and parse the remaining fields (or raw features).  To do this, complete the implemention of the `parse_point` function.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def parse_point(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
#       <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print(parse_point(raw_df.select('text').first()[0]))

In [ ]:
# TEST Extract features (3b)
assert_equal(parse_point(raw_df.select('text').first()[0])[:3], [(0, u'0'), (1, u'127'), (2, u'1')],
                  'incorrect implementation of parse_point')

### (3c) Extracting features continued

Next, we'll create a `parse_raw_df` function that creates a `label` column for the first value in a data point and a `features` column for the rest.  The `features` column will be created using `parse_point_udf`, which we've provided and is based on your `parse_point` function.  Note that to name your columns you should use [alias](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.alias).  You can split the `text` field in `raw_df` using [split](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split) and retrieve the first value of the resulting array with [getItem](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.getItem). Be sure to call [cast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast) to cast the column value to `double`. Your `parse_raw_df` function should also cache the DataFrame it returns.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with the appropriate code
from pyspark.sql.functions import udf, split
from pyspark.sql.types import ArrayType, StructType, StructField, LongType, StringType

parse_point_udf = udf(parse_point, ArrayType(StructType([StructField('_1', LongType()),
                                                         StructField('_2', StringType())])))

def parse_raw_df(raw_df):
    """Convert a DataFrame consisting of rows of comma separated text into labels and features.

    Args:
        raw_df (DataFrame with a 'text' column): DataFrame containing the raw comma separated data.

    Returns:
        DataFrame: A DataFrame with 'label' and 'features' columns.
    """
#     return raw_df.select(<FILL IN>)
#           .<FILL IN>
#           .<FILL IN>
#           .<FILL IN>
#           .<FILL IN>
#           .<FILL IN>
#           .cache()

  
# YOUR CODE HERE
raise NotImplementedError()

# # Parse the raw training DataFrame
# parsed_train_df = <FILL IN>  

# YOUR CODE HERE
raise NotImplementedError()

from pyspark.sql.functions import (explode, col)
num_categories = (parsed_train_df
                   .select(explode('features').alias('features'))
                   .distinct()
                   .select(col('features').getField('_1').alias('featureNumber'))
                   .groupBy('featureNumber')
                   .sum()
                   .orderBy('featureNumber')
                   .collect())

print(num_categories[2][1])

In [ ]:
# TEST Extract features (3c)
assert_true(parsed_train_df.is_cached, 'parse_raw_df should return a cached DataFrame')
assert_equal(num_categories[2][1], 1356, 'incorrect implementation of parse_point or parse_raw_df')


### (3d) Create an OHE dictionary from the dataset

Note that `parse_point` returns a data point in the format of a list of `(featureID, category)` tuples, which is the same format as the sample dataset studied in Parts 1 and 2 of this lab.  Using this observation, create an OHE dictionary from the parsed training data using the function implemented in Part (2c). Note that we will assume for simplicity that all features in our CTR dataset are categorical.

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# ctr_ohe_dict = <FILL IN>
# num_ctr_ohe_feats = <FILL IN>
# YOUR CODE HERE
raise NotImplementedError()

print(ctr_ohe_dict[(0, '4')])    

In [ ]:
# TEST Create an OHE dictionary from the dataset (3d)
assert_equal(num_ctr_ohe_feats, 215556, 'incorrect number of features in ctr_ohe_dict')
assert_true((0, '4') in ctr_ohe_dict, 'incorrect features in ctr_ohe_dict')

### (3e) Apply OHE to the dataset

Now let's use this OHE dictionary, by starting with the training data that we've parsed into `label` and `features` columns, to create one-hot-encoded features.  Recall that we created a function `ohe_udf_generator` that can create the UDF that we need to convert row into `features`.  Make sure that `ohe_train_df` contains a `label` and `features` column and is cached.

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with the appropriate code
# ohe_dict_broadcast = <FILL IN>
# ohe_dict_udf = <FILL IN>
# ohe_train_df = (parsed_train_df
#                   <FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

print(ohe_train_df.count())
print(ohe_train_df.take(1))

In [ ]:
# TEST Apply OHE to the dataset (3e)
assert_true('label' in ohe_train_df.columns and 'features' in ohe_train_df.columns, 'ohe_train_df should have label and features columns')
assert_true(ohe_train_df.is_cached, 'ohe_train_df should be cached')
num_nz = sum(parsed_train_df.rdd.map(lambda r: len(r[1])).take(5))
num_nz_alt = sum(ohe_train_df.rdd.map(lambda r: len(r[1].indices)).take(5))
assert_equal(num_nz, num_nz_alt, 'incorrect value for ohe_train_df')

### Visualization 1: Feature frequency

We will now visualize the number of times each of the 233,941 OHE features appears in the training data. We first compute the number of times each feature appears, then bucket the features by these counts.  The buckets are sized by powers of 2, so the first bucket corresponds to features that appear exactly once ( \\( \scriptsize 2^0 \\) ), the second to features that appear twice ( \\( \scriptsize 2^1 \\) ), the third to features that occur between three and four ( \\( \scriptsize 2^2 \\) ) times, the fifth bucket is five to eight ( \\( \scriptsize 2^3 \\) ) times and so on. The scatter plot below shows the logarithm of the bucket thresholds versus the logarithm of the number of features that have counts that fall in the buckets.

In [ ]:
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import log

get_indices = udf(lambda sv: list(map(int, sv.indices)), ArrayType(IntegerType()))
feature_counts = (ohe_train_df
                   .select(explode(get_indices('features')))
                   .groupBy('col')
                   .count()
                   .withColumn('bucket', log('count').cast('int'))
                   .groupBy('bucket')
                   .count()
                   .orderBy('bucket')
                   .collect())


In [ ]:
import matplotlib.pyplot as plt

x, y = zip(*feature_counts)
x, y = x, np.log(y)

def prepare_plot(xticks, yticks, figsize=(10.5, 6), hide_labels=False, grid_color='#999999',
                 grid_width=1.0):
    """Template for generating the plot layout."""
    plt.close()
    fig, ax = plt.subplots(figsize=figsize, facecolor='white', edgecolor='white')
    ax.axes.tick_params(labelcolor='#999999', labelsize='10')
    for axis, ticks in [(ax.get_xaxis(), xticks), (ax.get_yaxis(), yticks)]:
        axis.set_ticks_position('none')
        axis.set_ticks(ticks)
        axis.label.set_color('#999999')
        if hide_labels: axis.set_ticklabels([])
    plt.grid(color=grid_color, linewidth=grid_width, linestyle='-')
    map(lambda position: ax.spines[position].set_visible(False), ['bottom', 'top', 'left', 'right'])
    return fig, ax

# generate layout and plot data
fig, ax = prepare_plot(np.arange(0, 12, 1), np.arange(0, 14, 2))
ax.set_xlabel(r'$\log_e(bucketSize)$'), ax.set_ylabel(r'$\log_e(countInBucket)$')
plt.scatter(x, y, s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
display(fig)

### (3f) Handling unseen features

We naturally would like to repeat the process from Part (3e), to compute OHE features for the validation and test datasets.  However, we must be careful, as some categorical values will likely appear in new data that did not exist in the training data. To deal with this situation, update the `one_hot_encoding()` function from Part (1d) to ignore previously unseen categories, and then compute OHE features for the validation data.  Remember that you can parse a raw DataFrame using `parse_raw_df`.
> Note: you'll have to generate a new UDF using `ohe_udf_generator` so that the updated `one_hot_encoding` function is used.  And make sure to cache `ohe_validation_df`.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def one_hot_encoding(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted, and that the
        function handles missing features.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """

#     ohe_feature = <FILL IN>
#     return SparseVector(<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

# ohe_dict_missing_udf = <FILL IN>
# ohe_validation_df = (<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

ohe_validation_df.count()
display(ohe_validation_df) # replace with ohe_validate_df.show() if running outside of Databricks

In [ ]:
# TEST Handling unseen features (3f)
from pyspark.sql.functions import size, sum as sqlsum

assert_true(ohe_validation_df.is_cached, 'you need to cache ohe_validation_df')
num_nz_val = (ohe_validation_df
                .select(sqlsum(size(get_indices('features'))))
                .first()[0])

nz_expected = 367573
assert_equal(num_nz_val, nz_expected, 'incorrect number of features: Got {0}, expected {1}'.format(num_nz_val, nz_expected))

## Part 4: CTR prediction and logloss evaluation

### (4a) Logistic regression

We are now ready to train our first CTR classifier.  A natural classifier for this setting is logistic regression, since it models the probability of a click-through event rather than returning a simple binary "yes" or "no". Also, when working with rare events like clicking-through, probabilistic predictions are usually more accurate.

First use [LogisticRegression](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegression) from the pyspark.ml package to train a model using `ohe_train_df` with a given hyperparameter configuration.  `LogisticRegression.fit` returns a [LogisticRegressionModel](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegressionModel).  Next, we'll use the `LogisticRegressionModel.coefficients` and `LogisticRegressionModel.intercept` to print out some details of the model's parameters.  Note that these are the names of the object's attributes and should be called using a syntax like `model.coefficients` for a given `model`.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# Given hyperparameters
standardization = False
elastic_net_param = 0.0
reg_param = .01
max_iter = 20

from pyspark.ml.classification import LogisticRegression
# lr = (<FILL IN>)

# lr_model_basic = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print('intercept: {0}'.format(lr_model_basic.intercept))
print('length of coefficients: {0}'.format(len(lr_model_basic.coefficients)))
sorted_coefficients = sorted(lr_model_basic.coefficients)[:5]

In [ ]:
# TEST Logistic regression (4a)
assert_true(np.allclose(lr_model_basic.intercept,  -1.1870497039599432, atol=1e-2), 'incorrect value for model intercept')
assert_true(np.allclose(sorted_coefficients,
                           [-0.10347285277044568, -0.10296978958368273, -0.10296978958368273, -0.10296978958368273, -0.10296978958368273], atol=1e-2),
                           'incorrect value for model coefficients')

### (4b) Log loss
Throughout this lab, we will use log loss to evaluate the quality of models.  Log loss is defined as: \\[ \scriptsize \ell_{log}(p, y) = \begin{cases} -\log (p) & \text{if } y = 1 \\\ -\log(1-p) & \text{if } y = 0 \end{cases} \\] where \\( \scriptsize p\\) is a probability between 0 and 1 and \\( \scriptsize y\\) is a label of either 0 or 1. Log loss is a standard evaluation criterion when predicting rare-events such as click-through rate prediction.

Write a function `add_log_loss` for a DataFrame, and evaluate it on sample inputs.  This operation does not require a UDF.  You can perform a conditional branching with DataFrame columns using [when](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.when).

In [ ]:
# Some example data
example_log_loss_df = sqlContext.createDataFrame([(.5, 1), (.5, 0), (.99, 1), (.99, 0), (.01, 1),
                                                  (.01, 0), (1., 1), (.0, 1), (1., 0)], ['p', 'label'])
example_log_loss_df.show()

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
from pyspark.sql.functions import when, log, col
epsilon = 1e-16

def add_log_loss(df):
    """Computes and adds a 'log_loss' column to a DataFrame using 'p' and 'label' columns.

    Note:
        log(0) is undefined, so when p is 0 we add a small value (epsilon) to it and when
        p is 1 we subtract a small value (epsilon) from it.

    Args:
        df (DataFrame with 'p' and 'label' columns): A DataFrame with a probability column
            'p' and a 'label' column that corresponds to y in the log loss formula.

    Returns:
        DataFrame: A new DataFrame with an additional column called 'log_loss'
    """
#     return df.withColum(<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

add_log_loss(example_log_loss_df).show()

In [ ]:
# TEST Log loss (4b)
log_loss_values = add_log_loss(example_log_loss_df).select('log_loss').rdd.map(lambda r: r[0]).collect()
assert_true(np.allclose(log_loss_values[:-2],
                            [0.6931471805599451, 0.6931471805599451, 0.010050335853501338, 4.60517018598808,
                             4.605170185988081, 0.010050335853501338, -0.0], atol=1e-2), 'log loss is not correct')
assert_true(not(any(map(lambda x: x is None, log_loss_values[-2:]))),
                'log loss needs to bound p away from 0 and 1 by epsilon')

### (4c)  Baseline log loss

Next we will use the function we have in Part (4b) to compute the baseline log loss of the training data. A very simple yet natural baseline model is that we always make the same prediction regardless of datapoints, therefore the predicted value would be equal to the fraction of training points that correspond to click-through events (i.e., where the label is one). Compute this value (which is simply the mean of the training labels), and then use it to compute the training log loss for the baseline model.

> Note: you'll need to add a `p` column to the `ohe_train_df` DataFrame so that it can be used in your function from Part (4b).  To represent a constant value as a column you can use the [lit](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lit) function to wrap the value.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# Note that our dataset has a very high click-through rate by design
# In practice click-through rate can be one to two orders of magnitude lower

from pyspark.sql.functions import lit
# ctr_fraction = (<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()
print('Training class one fraction = {0:.3f}'.format(ctr_fraction))

# log_loss_tr_base = (<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()
print('Baseline Train Logloss = {0:.3f}\n'.format(log_loss_tr_base))

In [ ]:
# TEST Baseline log loss (4c)
expected_frac = 0.2339125
expected_log_loss = 0.5439608117656105
assert_true(np.allclose(ctr_fraction, expected_frac, atol=1e-2), 'incorrect value for ctr_fraction. Got {0}, expected {1}'.format(ctr_fraction, expected_frac))
assert_true(np.allclose(log_loss_tr_base, expected_log_loss, atol=1e-2), 'incorrect value for log_loss_tr_base. Got {0}, expected {1}'.format(log_loss_tr_base, expected_log_loss))

### (4d) Predict probability

In order to compute the log loss for the model we trained in Part (4a), we need to generate predictions from this model. Write a function that computes the raw linear prediction from this logistic regression model and then passes it through a [sigmoid function](http://en.wikipedia.org/wiki/Sigmoid_function) \\( \scriptsize \sigma(t) = (1+ e^{-t})^{-1} \\) to return the model's probabilistic prediction. Then compute probabilistic predictions on the training data.

Note that when incorporating an intercept into our predictions, we simply add the intercept to the value of the prediction obtained from the weights and features.  Alternatively, if the intercept was included as the first weight, we would need to add a corresponding feature to our data where the feature has the value one.  This is not the case here.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
from pyspark.sql.types import DoubleType
from math import exp #  exp(-t) = e^-t

def add_probability(df, model):
    """Adds a probability column ('p') to a DataFrame given a model"""
    coefficients_broadcast = sc.broadcast(model.coefficients)
    intercept = model.intercept

    def get_p(features):
        """Calculate the probability for an observation given a list of features.

        Note:
            We'll bound our raw prediction between 20 and -20 for numerical purposes.

        Args:
            features: the features

        Returns:
            float: A probability between 0 and 1.
        """
#         # Compute the raw value
#         raw_prediction = <FILL IN>
#         # Bound the raw value between 20 and -20
#         raw_prediction = <FILL IN>
#         # Return the probability
#         <FILL IN>
        
# YOUR CODE HERE
raise NotImplementedError()

    get_p_udf = udf(get_p, DoubleType())
    return df.withColumn('p', get_p_udf('features'))

add_probability_model_basic = lambda df: add_probability(df, lr_model_basic)
training_predictions = add_probability_model_basic(ohe_train_df).cache()

training_predictions.show(5)

In [ ]:
# TEST Predicted probability (4d)
expected = 18746.356946150863
got = training_predictions.selectExpr('sum(p)').first()[0]
assert_true(np.allclose(got, expected, atol=1e-2),
                'incorrect value for training_predictions. Got {0}, expected {1}'.format(got, expected))

### (4e) Evaluate the model

We are now ready to evaluate the performance of the model we trained in Part (4a). To do this, first write a general function that takes a model and a DataFrame as its input, and outputs the log loss. Note that the log loss for multiple observations should be the mean of all the individual log losses. Then run this function on the OHE training data, and compare the result with the baseline log loss.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def evaluate_results(df, model, baseline=None):
    """Calculates the log loss for the data given the model.

    Note:
        If baseline has a value the probability should be set to baseline before
        the log loss is calculated.  Otherwise, use add_probability to add the
        appropriate probabilities to the DataFrame.

    Args:
        df (DataFrame with 'label' and 'features' columns): A DataFrame containing
            labels and features.
        model (LogisticRegressionModel): A trained logistic regression model. This
            can be None if baseline is set.
        baseline (float): A baseline probability to use for the log loss calculation.

    Returns:
        float: Log loss for the data.
    """
    
#     with_probability_df = <FILL IN>
#     with_log_loss_df = <FILL IN>
#     log_loss = <FILL IN>
#     return log_loss

# YOUR CODE HERE
raise NotImplementedError()


log_loss_train_model_basic = evaluate_results(ohe_train_df, lr_model_basic)
print('OHE Features Train Logloss:\n\tBaseline = {0:.3f}\n\tLogReg = {1:.3f}'
       .format(log_loss_tr_base, log_loss_train_model_basic))

In [ ]:
# TEST Evaluate the model (4e)
expected_log_loss = 0.48971226940239815
assert_true(np.allclose(log_loss_train_model_basic, expected_log_loss, atol=1e-2),
                'incorrect value for log_loss_train_model_basic. Got {0}, expected {1}'.format(log_loss_train_model_basic, expected_log_loss))
expected_res = 0.6931471805600546
res = evaluate_results(ohe_train_df, None,  0.5)
assert_true(np.allclose(res, expected_res, atol=1e-2),
                'evaluate_results needs to handle baseline models. Got {0}, expected {1}'.format(res, expected_res))

### (4f) Log loss of validation dataset

Next, use the `evaluate_results` function and compute the log loss of validation dataset for both the baseline and logistic regression models. Notably, the baseline model for the validation dataset should still be based on the label fraction from the training dataset.

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# log_loss_val_base = <FILL IN>

# log_loss_val_l_r0 = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print(('OHE Features Validation Logloss:\n\tBaseline = {0:.3f}\n\tLogReg = {1:.3f}'
       .format(log_loss_val_base, log_loss_val_l_r0)))

In [ ]:
# TEST Validation log loss (4f)
expected_val_base = 0.6344644324013423
assert_true(np.allclose(log_loss_val_base, expected_val_base, atol=1e-2),
                'incorrect value for log_loss_val_base. Got {0}, expected {1}'.format(log_loss_val_base, expected_val_base))
expected_val_model_basic = 0.5793520014798194
assert_true(np.allclose(log_loss_val_l_r0, expected_val_model_basic, atol=1e-2),
                'incorrect value for log_loss_val_l_r0. Got {0}, expected {1}'.format(log_loss_val_l_r0, expected_val_model_basic))

### Visualization 2: ROC curve

We will now visualize the performance of our model.  We generate a plot called the ROC curve.  The ROC curve shows us the trade-off between the false positive rate and true positive rate, as we liberalizing the threshold required for a positive prediction.  The performance of a random model is represented by the dashed line in the plot.

In [ ]:
labels_and_scores = add_probability_model_basic(ohe_validation_df).select('label', 'p')
labels_and_weights = labels_and_scores.collect()
labels_and_weights.sort(key=lambda x: x[1], reverse=True)
labels_by_weight = np.array([k for (k, v) in labels_and_weights])

length = labels_by_weight.size
true_positives = labels_by_weight.cumsum()
num_positive = true_positives[-1]
false_positives = np.arange(1.0, length + 1, 1.) - true_positives

true_positive_rate = true_positives / num_positive
false_positive_rate = false_positives / (length - num_positive)

# Generate layout and plot data
fig, ax = prepare_plot(np.arange(0., 1.1, 0.1), np.arange(0., 1.1, 0.1))
ax.set_xlim(-.05, 1.05), ax.set_ylim(-.05, 1.05)
ax.set_ylabel('True Positive Rate (Sensitivity)')
ax.set_xlabel('False Positive Rate (1 - Specificity)')
plt.plot(false_positive_rate, true_positive_rate, color='#8cbfd0', linestyle='-', linewidth=3.)
plt.plot((0., 1.), (0., 1.), linestyle='--', color='#d6ebf2', linewidth=2.)  # Baseline model
display(fig)

## Part 5: Reduce features' dimension via feature hashing

### (5a) Hash function

As we just saw, using an one-hot-encoding featurization can yield a model with good statistical accuracy.  However, the number of distinct categories across all features is quite large -- recall that we observed 233K categories in the training data in Part (3c).  Moreover, the full training dataset includes more than 33M distinct categories, and the training dataset itself is just a small subset of labeled data in real world.  Hence, featurizing via an one-hot-encoding representation could lead to a very large feature vector. To reduce the dimensionality of the feature space, we will use feature hashing.

Below is a hash function that we will use for this part of the lab.  We will first use this hash function with the three sample data points from Part (1a) to gain some intuition.  Implement the following code to hash those three sample points using two different values for `numBuckets` and observe the resulting hashed feature dictionaries.

In [ ]:
from collections import defaultdict
import functools 
import hashlib

def hash_function(raw_feats, num_buckets, print_mapping=False):
    """Calculate a feature dictionary for an observation's features based on hashing.

    Note:
        Use print_mapping=True for debug purposes and to better understand how the hashing works.

    Args:
        raw_feats (list of (int, str)): A list of features for an observation.  Represented as
            (featureID, value) tuples.
        num_buckets (int): Number of buckets to use as features.
        print_mapping (bool, optional): If true, the mappings of featureString to index will be
            printed.

    Returns:
        dict of int to float:  The keys will be integers which represent the buckets that the
            features have been hashed to.  The value for a given key will contain the count of the
            (featureID, value) tuples that have hashed to that key.
    """
    mapping = { category + ':' + str(ind):
                int(int(hashlib.md5((category + ':' + str(ind)).encode('utf-8')).hexdigest(), 16) % num_buckets)
                for ind, category in raw_feats}
    if(print_mapping): print(mapping)

    def map_update(l, r):
        l[r] += 1.0
        return l

    sparse_features = functools.reduce(map_update, mapping.values(), defaultdict(float))
    return dict(sparse_features)

# Reminder of the sample values:
# sample_one = [(0, 'mouse'), (1, 'black')]
# sample_two = [(0, 'cat'), (1, 'tabby'), (2, 'mouse')]
# sample_three =  [(0, 'bear'), (1, 'black'), (2, 'salmon')]

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# # Use four buckets
# samp_one_four_buckets = hash_function(sample_one, < FILL IN >, True)
# samp_two_four_buckets = hash_function(sample_two, < FILL IN >, True)
# samp_three_four_buckets = hash_function(sample_three, < FILL IN >, True)

# # Use one hundred buckets
# samp_one_hundred_buckets = hash_function(sample_one, < FILL IN >, True)
# samp_two_hundred_buckets = hash_function(sample_two, < FILL IN >, True)
# samp_three_hundred_buckets = hash_function(sample_three, < FILL IN >, True)

# YOUR CODE HERE
raise NotImplementedError()

print('\n\t\t 4 Buckets \t\t\t 100 Buckets')
print('Sample One:\t {0}\t\t\t {1}'.format(samp_one_four_buckets, samp_one_hundred_buckets))
print('Sample Two:\t {0}\t\t {1}'.format(samp_two_four_buckets, samp_two_hundred_buckets))
print('Sample Three:\t {0}\t {1}'.format(samp_three_four_buckets, samp_three_hundred_buckets))

In [ ]:
# TEST Hash function (5a)
assert_equal(samp_one_four_buckets, {3: 2.0}, 'incorrect value for samp_one_four_buckets')
assert_equal(samp_three_hundred_buckets, {80: 1.0, 82: 1.0, 51: 1.0},
                  'incorrect value for samp_three_hundred_buckets')

### (5b) Create hashed features

Next, we will use this hash function to create hashed features for our CTR datasets. Use the given UDF to create a function that takes in a DataFrame and returns both labels and hashed features.  Then use it to create new training, validation and test datasets with hashed features.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
from pyspark.ml.linalg import Vectors
num_hash_buckets = 2 ** 15

# UDF that returns a vector of hashed features given an Array of tuples
tuples_to_hash_features_udf = udf(lambda x: Vectors.sparse(num_hash_buckets, hash_function(x, num_hash_buckets)), VectorUDT())

def add_hashed_features(df):
    """Return a DataFrame with labels and hashed features.

    Note:
        Make sure you cache the DataFrame that you are returning.

    Args:
        df (DataFrame with 'label' and 'features' column): A DataFrame containing labels and the features to be hashed.

    Returns:
        DataFrame: A DataFrame with a 'label' column and a 'features' column that contains a
            SparseVector of hashed features.
    """
#     return     <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

# hash_train_df = <FILL IN>
# hash_validation_df = <FILL IN>
# hash_test_df = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

hash_train_df.show()

In [ ]:
# TEST Creating hashed features (5b)
hash_train_df_feature_sum = sum(hash_train_df
                                  .rdd
                                  .map(lambda r: sum(r[1].indices))
                                  .take(10))
hash_validation_df_feature_sum = sum(hash_validation_df
                                       .rdd
                                       .map(lambda r: sum(r[1].indices))
                                       .take(10))
hash_test_df_feature_sum = sum(hash_test_df
                                 .rdd
                                 .map(lambda r: sum(r[1].indices))
                                 .take(10))

expected_train_sum = 6333443
assert_equal(hash_train_df_feature_sum, expected_train_sum,
                  'incorrect number of features in hash_train_df. Got {0}, expected {1}'.format(hash_train_df_feature_sum, expected_train_sum))

expected_validation_sum = 6340030
assert_equal(hash_validation_df_feature_sum, expected_validation_sum,
                  'incorrect number of features in hash_validation_df. Got {0}, expected {1}'.format(hash_validation_df_feature_sum, expected_validation_sum))

### (5c) Sparsity

Since we only have 33K hashed features versus 233K OHE features, we could expect our OHE features to be sparser. Verify this hypothesis by computing the average sparsity of the OHE and the hashed training datasets.

Note that if you have a `SparseVector` named `sparse`, calling `len(sparse)` returns the total number of features, not the number features with entries.  `SparseVector` objects have the attributes `indices` and `values` that contain information about non-zero features.  Continuing with our example, these can be accessed using `sparse.indices` and `sparse.values`, respectively.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
def vector_feature_sparsity(sparse_vector):
    """Calculates the sparsity of a SparseVector.

    Args:
        sparse_vector (SparseVector): The vector containing the features.

    Returns:
        float: The ratio of features found in the vector to the total number of features.
    """
#     return     <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

a_sparse_vector = Vectors.sparse(5, {0: 1.0, 3: 1.0})
a_sparse_vector_sparsity = vector_feature_sparsity(a_sparse_vector)
print('This vector should have sparsity 2/5 or .4.')
print('Sparsity = {0:.2f}.'.format(a_sparse_vector_sparsity))

In [ ]:
# TEST Sparsity (5c)
assert_equal(a_sparse_vector_sparsity, .4,
                'incorrect value for a_sparse_vector_sparsity')

### (5d) Sparsity continued

Now we have a function to calculate vector sparsity, we'll wrap it up in a UDF and apply it to the entire DataFrame to obtain the average sparsity of features in that DataFrame.  We'll use this function to calculate the average sparsity of both the one-hot-encoded training DataFrame and  the hashed training DataFrame.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
feature_sparsity_udf = udf(vector_feature_sparsity, DoubleType())

def get_sparsity(df):
    """Calculates the average sparsity for the features in a DataFrame.

    Args:
        df (DataFrame with 'features' column): A DataFrame with sparse features.

    Returns:
        float: The average feature sparsity.
    """
#     return (<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

# average_sparsity_ohe = <FILL IN>
# average_sparsity_hash = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print('Average OHE Sparsity: {0:.7e}'.format(average_sparsity_ohe))
print('Average Hash Sparsity: {0:.7e}'.format(average_sparsity_hash))

In [ ]:
# TEST Sparsity (5d)
expected_ohe = 1.8092746e-04
assert_true(np.allclose(average_sparsity_ohe, expected_ohe, atol=1e-2),
                'incorrect value for average_sparsity_ohe. Got {0}, expected {1}'.format(average_sparsity_ohe, expected_ohe))
expected_hash = 1.1895943e-03
assert_true(np.allclose(average_sparsity_hash, expected_hash, atol=1e-2),
                'incorrect value for average_sparsity_hash. Got {0}, expected {1}'.format(average_sparsity_hash, expected_hash))

### (5e) Logistic model with hashed features

Now let's train a logistic regression model using the hashed training features. Use the given hyperparameters, train and fit the model, then evaluate the log loss on the training set.

In [ ]:
# TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# Given hyperparameters
standardization = False
elastic_net_param = 0.7
reg_param = .001
max_iter = 20

# lr_hash = (<FILL IN>)

# lr_model_hashed = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print('intercept: {0}'.format(lr_model_hashed.intercept))
print(len(lr_model_hashed.coefficients))

log_loss_train_model_hashed = evaluate_results(hash_train_df, lr_model_hashed)
print(('OHE Features Train Logloss:\n\tBaseline = {0:.3f}\n\thashed = {1:.3f}'
       .format(log_loss_tr_base, log_loss_train_model_hashed)))

In [ ]:
# TEST Logistic model with hashed features (5e)
expected =  0.481478172974873
assert_true(np.allclose(log_loss_train_model_hashed, expected, atol=1e-2),
                'incorrect value for log_loss_train_model_hashed. Got {0}, expected {1}'.format(log_loss_train_model_hashed, expected))

### (5f) Evaluate the performance on the test set

Finally, we will evaluate the model from Part (5e) on the test set.  Compare the resulting log loss with the baseline log loss on the test set, which can be computed in the same way where the validation log loss was computed in Part (4f).

In [ ]:
# # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
# # Log loss for the best model from (5e)
# log_loss_test = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

# ## Log loss for the baseline model
# Use train lables to calculate this as test labels will not be available
# print('Class one fraction for test data: {0}'.format(ctr_fraction))
# log_loss_test_baseline = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

print(('Hashed Features Test Log Loss:\n\tBaseline = {0:.3f}\n\tLogReg = {1:.3f}'
       .format(log_loss_test_baseline, log_loss_test)))

In [ ]:
# TEST Evaluate on the test set (5f)
expected_test_baseline = 0.6443111314185229
assert_true(np.allclose(log_loss_test_baseline, expected_test_baseline, atol=1e-2),
                'incorrect value for log_loss_test_baseline. Got {0}, expected {1}'.format(log_loss_test_baseline, expected_test_baseline))
expected_test = 0.5748571343610652
assert_true(np.allclose(log_loss_test, expected_test, atol=1e-2),
                'incorrect value for log_loss_test. Got {0}, expected {1}'.format(log_loss_test, expected_test))